# Pre processing of the data
## Imports

In [1]:
!pip install - -upgrade pip
!pip install pycountry


/bin/bash: pip: command not found
/bin/bash: pip: command not found


In [2]:
import pandas as pd
import numpy as np
import pycountry
import matplotlib.pyplot as plt
import plotly.express as px


In [3]:
# df = pd.read_csv(
#    'hfactory_magic_folders/groups/14_none_shared_workspace/datasets/train-data.csv', sep=';')


In [4]:
df = pd.read_csv(
    'datasets/train-data.csv', sep=';')

In [5]:
df.head(1)


,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4
0,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,Division-3,Customer Segmentation-3,Strategic Product Family-12,NaN,may-aug 2021,0,0,0,0


Let's replace the NaN values in "Product Life cycel status" that in reality correspond to the category "active" :


In [6]:
df['Product Life cycel status'] = df['Product Life cycel status'].fillna(
    'active')


In order to merge, let's split the "Date" column in the train data set into 2 columns : a "year" column and a "Third" one.
There is one observation without date, let's drop it.

In [7]:
df = df.dropna(axis=0, subset='Date')


Create a Year column, and then a Third column:

In [8]:
df['Year'] = df['Date'].str.slice(start=-4).astype(int)

df['Third'] = df['Date'].str.slice(
    stop=-5).replace({'jan-apr': 'T1', 'sep-dec': 'T3', 'may-aug': 'T2', 'may-jul': 'T2'})

# We now rearrange and drop the obsolete 'date' column
#df = df.iloc[:, list(range(0, 18)) + [20, 19, 18]].drop(columns=['Date'])


In [9]:
df.head(1)


,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,...,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4,Year,Third
0,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,...,Customer Segmentation-3,Strategic Product Family-12,active,may-aug 2021,0,0,0,0,2021,T2


In [10]:
df['Country']


0          DE
1          CN
2          GB
3          AU
4          ES
           ..
1597071    FR
1597072    CN
1597073    TH
1597074    US
1597075    FR
Name: Country, Length: 1597076, dtype: object

We want to join the extra data sets and the main set on countries as well. Let's make sure the ISOs are standards.

In [11]:
df['Country'] = df['Country'].replace({'Id': 'ID'})


## Processing of the extra datasets

In [12]:
gscpi_data = pd.read_csv(
    "datasets/extra-dataset/GSCPI_data.csv")
lpi_data = pd.read_csv(
    "datasets/extra-dataset/LPIextend.csv")
economic_data = pd.read_csv(
    "datasets/extra-dataset/worldbank_economic_data.csv")
inflation_data = pd.read_csv(
    "datasets/extra-dataset/worldbank_inflation_data.csv")


Some of the naming isn't consistent, let's change it (arbitrarly) so no error comes up later.

In [13]:
d_LPI = {'Taiwan': 'Taiwan, Province of China',
         'South Korea': 'Korea, Republic of',
         'Vietnam': 'Viet Nam',
         'Brunei': 'Brunei Darussalam',
         'Lao PDR': "Lao People's Democratic Republic"}


d_Eco = {'Taiwan, China': 'Taiwan, Province of China',
         'Hong Kong SAR, China': 'Hong Kong',
         'Korea, Rep.': 'Korea, Republic of',
         'Vietnam': 'Viet Nam',
         'Lao, PDR': "Lao People's Democratic Republic"}


d_Inf = {'Taiwan, China': 'Taiwan, Province of China',
         'Hong Kong SAR, China': 'Hong Kong',
         'Korea, Rep.': 'Korea, Republic of',
         'Vietnam': 'Viet Nam',
         'Lao, PDR': "Lao People's Democratic Republic"}

lpi_data['Country'] = lpi_data['Country'].replace(d_LPI)
inflation_data['Country'] = inflation_data['Country'].replace(d_Inf)
economic_data['Country'] = economic_data['Country'].replace(d_Eco)


The goal is now to make sure we have a map between our train set's country codes onto the extra datasets countries.

In [14]:
def split_year_month_column(dataframe, column_name):

    if column_name not in dataframe.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    dataframe[['Year', 'Month']] = dataframe[column_name].str.split(
        '-', expand=True)

    dataframe['Year'] = pd.to_numeric(dataframe['Year'])
    dataframe['Month'] = pd.to_numeric(dataframe['Month'])

    month_to_quarter = {1: 'T1', 2: 'T1', 3: 'T1', 4: 'T1',
                        5: 'T2', 6: 'T2', 7: 'T2', 8: 'T2',
                        9: 'T3', 10: 'T3', 11: 'T3', 12: 'T3'}

    dataframe['Third'] = dataframe['Month'].map(month_to_quarter)

    return dataframe


gscpi_data = split_year_month_column(gscpi_data, "Year-Month")
inflation_data = split_year_month_column(inflation_data, "Year-Month")


In [15]:
gscpi_data.drop(["Year-Month", "Month"], axis=1, inplace=True)
inflation_data.drop(["Year-Month", "Month"], axis=1, inplace=True)


In [16]:
def add_iso_country_column(
        dataframe,
        country_column_name,
        iso_column_name='ISO_Code'):

    iso_codes = []
    for country_name in dataframe[country_column_name]:
        try:
            country = pycountry.countries.get(name=country_name)
            iso_codes.append(country.alpha_2)
        except AttributeError:
            iso_codes.append(None)

    dataframe[iso_column_name] = iso_codes


add_iso_country_column(lpi_data, 'Country', 'ISO_Code')
add_iso_country_column(inflation_data, 'Country', 'ISO_Code')
add_iso_country_column(economic_data, 'Country', 'ISO_Code')


## Merger of the data sets


For the LPI data, we keep most of the quantitative data, excluding the ranks.


In [17]:
lpi_data = lpi_data[['population (2023)', 'netChange', 'growthRate', 'worldPercentage',
                     'Customs Score', 'Infrastructure Score', 'International Shipments Score',
                     'Logistics Competence and Quality Score', 'Timeliness Score',
                     'Tracking and Tracing Score', 'ISO_Code']]


In [18]:
df = df.join(lpi_data.set_index('ISO_Code'), on='Country', how='left')


Let's merge the GSCPI dataset. For simplicity, we compute the mean of the GSCPI for each Third.


In [19]:
gscpi_data = gscpi_data.groupby(by=["Year", "Third"]).mean()
merged_df = pd.merge(df, gscpi_data, on=['Year', 'Third'], how='left')


Let's merge the economic data set onto the main dataset:


In [20]:
economic_data = economic_data.drop(columns=['Country'])

economic_data.rename(columns={'ISO_Code': 'Country'}, inplace=True)


In [21]:
df = pd.merge(df, economic_data, on=['Year', 'Country'], how='left')


Let's do the same with the inflation data

In [22]:
inflation_data = inflation_data.drop(columns=['Country'])

inflation_data.rename(columns={'ISO_Code': 'Country'}, inplace=True)


Again, we take the mean of the various indices over each third for each country.

In [23]:
inflation_data = inflation_data.groupby(by=["Year", "Third", 'Country']).mean()


In [24]:
df = pd.merge(df, inflation_data, on=['Year', 'Country', 'Third'], how='left')


## Handling of the NaN and extreme data

The variable "Month 1" contains around 170 000 missing values. The value 0 is immensely predominant.

In [25]:
df["Month 1"] = df["Month 1"].fillna(0)


## Let's continue this in another notebook

In [26]:
df.to_csv('datasets/Merged_train_yann.csv', sep=';')

In [ ]:
#import pandas as pd
#pd.read_csv("datasets/X_test_processed.csv", delimiter=";").head()